# How to Track the Properties of Real Packets

**TARDIS** has the functionality to track the properties of the *RPackets* that are generated when running the Simulation. The `rpacket_tracker` can track all the interactions a packet undergoes & thus keeps a track of the various properties, a packet may have.<br>Currently, the `rpacket_tracker` tracks the properties of all the rpackets in the *Last Iteration of the Simulation*. It generates a `List` that contains the individual instances of `RPacketCollection`{`Numba JITClass`}, for storing all the interaction properties as listed below.

<html>
    <body>
        The properties that are tracked are as follows :
        <ol>
            <li><b>index</b> - Index of the Packet</li>
            <li><b>seed</b> - Seed of the Packet</li>
            <li><b>status</b> - Current Status for the Packet Interaction</li>
            <li><b>r</b> - Radius of the Current Shell</li> 
            <li><b>nu</b> - Packet's Frequency</li> 
            <li><b>mu</b> - Propagation Direction of the Packet (cosine of the angle the packet’s path makes with the radial direction)</li>
            <li><b>energy</b> - Energy of the Packet</li>
            <li><b>shell_id</b> - Current Shell Id where the Packet is present</li>
            <li><b>interaction_type</b> - Last Interaction type of the packet</li>
        </ol>
    </body>
</html>

The data can be obtained in two ways i.e. `rpacket_tracker` and `rpacket_tracker_df`. The `rpacket_tracker` stores all the data for the interaction of the packets in a `list`, so it needs to accessed with a `list index` for each property for a particular `rpacket`. `rpacket_tracker_df` stores the data in a dataframe. Examples for the same are shown as follows. 

## How to Setup the Tracking for the RPackets?

**TARDIS**' `rpacket_tracker` is configured via the `YAML` file. This functionality of tracking the packets is turned **off**, by default. This is due to that fact that using this property, may slow down the execution time for the Simulation. An example configuration can be seen below for setting up the *tracking*:

```yaml
... 
montecarlo:
...
tracking:
    track_rpacket: true
```

The `montecarlo` section of the **YAML** file now has a `tracking` sub section which holds the configuration properties for the `track_rpacket` & the `initial_array_length` (discussed later in the tutorial).

Let us see, the new `rpacket_tracker` in action.

In [ ]:
from tardis.io.configuration.config_reader import Configuration

In [ ]:
# Reading the Configuration stored in `tardis_config_packet_tracking.yml` into config

config = Configuration.from_yaml("tardis_example.yml")

In [ ]:
# Checking the `tracking` section via the Schema

config["montecarlo"]["tracking"]

In [ ]:
# Setting `r_packet_tracking` to True to turn on the Tracking 

config["montecarlo"]["tracking"]["track_rpacket"] = True

In [ ]:
config["montecarlo"]["tracking"]

In [ ]:
from tardis import run_tardis
from tardis.io.atom_data.util import download_atom_data

In [ ]:
download_atom_data('kurucz_cd23_chianti_H_He')

In [ ]:
# Running the simulation from the config

sim = run_tardis(config, show_convergence_plots=False, show_progress_bars=False)

Now, the `tracked` properties can be accessed via the `rpacket_tracker` attribute of the `sim.transport` object. 

In [ ]:
type(sim.transport.transport_state.rpacket_tracker)

It can be seen from the above code, that the `sim.transport.rpacket_tracker` is an instance of the `List` specifically *Numba Typed List*. The `RPacketCollection` class has the following structure for the properties : {More information in the **TARDIS API** for `RPacketCollection` class}

```python
# Basic structure for the RPacketCollection Class
class RPacketCollection:
   # Properties
    index
    seed
    status
    r
    nu
    mu
    energy
    shell_id
    interaction_type
```

In [ ]:
len(sim.transport.transport_state.rpacket_tracker)

### Accessing properties through `rpacket_tracker`

To access these different properties, we may consider the following examples for the `rpacket_tracker`:
<br>In this Example, we are trying to access the properties of the packet at index `10`.<br>In a similar way, we can check for any property for any packet in the range of packets for the last iteration.

#### Accessing the `index` property for the packet with index = 10 :

In [ ]:
sim.transport.transport_state.rpacket_tracker[10].index

#### Accessing the `seed` property for the packet with index = 10 :

In [ ]:
sim.transport.transport_state.rpacket_tracker[10].seed

#### Accessing the `status` property for the packet with index = 10 :

In [ ]:
sim.transport.transport_state.rpacket_tracker[10].status

Thus, all other properties (`r`, `nu`, `mu`, `energy`, `shell_id`,`interaction_type`) can be accessed accordingly.

We can also see the total number of interactions of index `10` packet under went, with the following example:

In [ ]:
len(sim.transport.transport_state.rpacket_tracker[10].shell_id)

<div class="alert alert-warning">

Warning

If we  try to access `sim.transport.rpacket_tracker` property when we have the `track_rpacket` property in the `tracking` subsection of `montecarlo` config, turned off as follows `config["montecarlo"]["tracking"]["track_rpacket"] = False`, it will return `None`.  Error will be raised if we try to access the properties i.e. `seed`, `index`, etc.
</div>

<div class="alert alert-info">

Note
    
When we initialise the `RPacketCollection()` class, the properties arrays {`index`, `seed`, `status`, etc} are allocated certain length based on the `initial_array_length` parameter that can be set via the `initial_array_length` property under `montecarlo -> tracking` section of the configuration. The default size of the array is `10`. This variable is important as the number of interactions a packet may have is variable, thus we need to allocate space dynamically. This variable is used to compute the size and expand the array such that the properties are able to hold these values for the packet interaction. Higher number, allocates more space initially leading to lesser times the arrays expands and vice versa. It can be set in the following manner `config["montecarlo"]["tracking"]["initial_array_length"] = {value}`.
</div>

### Accessing properties with `rpacket_tracker_df`

Also, the `tracked` properties can be accessed via the `rpacket_tracker_df` attribute of the `sim.transport` object. 

In [ ]:
type(sim.transport.transport_state.rpacket_tracker_df)

Here, all the properties of the rpackets discussed above are stored as a `Pandas.Dataframe` object. This makes accessing the properties of any rpacket more convenient and in a more organized way.

<html>
    <body>
        The Basic structure of the Dataframe is as follows:<br><br>
        Index Columns:
        <ol>
            <li><b>index</b> - Indicates the index number of the packet.</li>
            <li><b>step</b> - It is the step number that any particular packet is on, as it moves through the ejecta.</li>
        </ol>
        Properties:<br><br>
        These following properties are present as columns in the Dataframe. These are the same properties that were present in the <b>rpacket_tracker</b>
        <ol>
            <li><b>status</b></li>
            <li><b>seed</b></li>
            <li><b>r</b></li>
            <li><b>nu</b></li>
            <li><b>mu</b></li>
            <li><b>energy</b></li>
            <li><b>shell_id</b></li>
            <li><b>interaction_type</b></li>
        </ol>
    </body>
</html>

The entire dataframe, containing all the properties mentioned above, can be fetched as:

In [ ]:
sim.transport.transport_state.rpacket_tracker_df

To access these different properties in the dataframe, we may consider the following examples for the `rpacket_tracker_df`:
<br>In this Example, we are trying to access the properties of the packet at index `10`.<br>In a similar way, we can check for any property for any packet in the range of packets for the last iteration.

#### Accessing all the properties for the packet with index = 10:

In [ ]:
sim.transport.transport_state.rpacket_tracker_df.loc[10]

#### Accessing the `energy` property for the packet with index = 10:


In [ ]:
sim.transport.transport_state.rpacket_tracker_df.loc[10]["energy"]

The above command returns a Pandas Series for the packet `#10` across different steps. To access any property at a particular step (for example at step `5`), the following commands can be used.

In [ ]:
sim.transport.transport_state.rpacket_tracker_df.loc[10,5]["energy"]

In [ ]:
sim.transport.transport_state.rpacket_tracker_df.loc[10]["energy"][5]

Thus, all other properties (`status`, `seed`, `r`, `nu`, `mu`, `shell_id`,`interaction_type`) can be accessed accordingly.